In [1]:
import pandas as pd
import os

In [2]:
pd.set_option('display.float_format', '{:,.2f}'.format)

In [3]:
directory_files = os.listdir('files')

In [16]:
def parse_file_type(file_key, file_extension):
    files = {}
    for file_name in directory_files:
        if file_key in file_name:
            if file_name[-5:] == file_extension:
                df = pd.read_excel(os.path.join('files', file_name), 'Sheet1')
                df['file_name'] = file_name
                files[file_name] = df
    return files

In [17]:
bene_files = parse_file_type('Beneficiary_Summary_File', '.xlsx')

In [18]:
bene_files

{'DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx':          DESYNPUF_ID  BENE_BIRTH_DT  BENE_DEATH_DT  BENE_SEX_IDENT_CD  \
 0   00013D2EFD8E45D1       19230501            nan                  1   
 1   00016F745862898F       19430101            nan                  1   
 2   0001FDD721E223DC       19360901            nan                  2   
 3   00021CA6FF03E670       19410601            nan                  1   
 4   00024B3D2352D2D0       19360801            nan                  1   
 5   0002DAE1C81CC70D       19431001            nan                  1   
 6   0002F28CE057345B       19220701            nan                  1   
 7   000308435E3E5B76       19350901            nan                  1   
 8   000345A39D4157C9       19760901            nan                  2   
 9   00036A21B65B0206       19381001            nan                  2   
 10  000489E7EAAD463F       19340201            nan                  2   
 11  00048EF1F4791C68       19290601            nan        

In [19]:
bene_df = pd.concat(bene_files.values())

In [20]:
bene_df

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR,file_name
0,00013D2EFD8E45D1,19230501,nan,1,1,0,26,950,12,12,...,0,0,0,50,10,0,0,0,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx
1,00016F745862898F,19430101,nan,1,1,0,39,230,12,12,...,0,0,0,0,0,0,700,240,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx
2,0001FDD721E223DC,19360901,nan,2,1,0,39,280,12,12,...,0,0,0,0,0,0,0,0,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx
3,00021CA6FF03E670,19410601,nan,1,5,0,6,290,0,0,...,0,0,0,0,0,0,0,0,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx
4,00024B3D2352D2D0,19360801,nan,1,1,0,52,590,12,12,...,0,0,0,30,40,0,220,80,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx
5,0002DAE1C81CC70D,19431001,nan,1,2,0,33,400,0,0,...,0,0,0,0,0,0,0,0,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx
6,0002F28CE057345B,19220701,nan,1,1,0,39,270,12,12,...,0,0,0,1010,270,0,3330,940,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx
7,000308435E3E5B76,19350901,nan,1,1,0,24,680,10,10,...,0,0,0,150,160,0,870,340,80,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx
8,000345A39D4157C9,19760901,nan,2,1,0,23,810,0,0,...,0,0,0,0,0,0,0,0,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx
9,00036A21B65B0206,19381001,nan,2,2,0,1,570,12,12,...,0,0,0,0,0,0,0,0,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx


In [21]:
diabetes = bene_df.SP_DIABETES.value_counts()

In [22]:
def parse_field(x):
    return x.split('_')[2]

In [23]:
file_year = bene_df.file_name.map(parse_field)

In [24]:
file_year.value_counts() # Matches page 14 of the SynPUF_Codebook

2008    2000
2010    2000
2009    2000
Name: file_name, dtype: int64

In [25]:
bene_df['file_year'] = file_year

In [26]:
bene_df.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR,file_name,file_year
0,00013D2EFD8E45D1,19230501,nan,1,1,0,26,950,12,12,...,0,0,50,10,0,0,0,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx,2008
1,00016F745862898F,19430101,nan,1,1,0,39,230,12,12,...,0,0,0,0,0,700,240,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx,2008
2,0001FDD721E223DC,19360901,nan,2,1,0,39,280,12,12,...,0,0,0,0,0,0,0,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx,2008
3,00021CA6FF03E670,19410601,nan,1,5,0,6,290,0,0,...,0,0,0,0,0,0,0,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx,2008
4,00024B3D2352D2D0,19360801,nan,1,1,0,52,590,12,12,...,0,0,30,40,0,220,80,0,DE1_0_2008_Beneficiary_Summary_File_Sample_1.xlsx,2008


In [27]:
bene_df.dtypes

DESYNPUF_ID                  object
BENE_BIRTH_DT                 int64
BENE_DEATH_DT               float64
BENE_SEX_IDENT_CD             int64
BENE_RACE_CD                  int64
BENE_ESRD_IND                object
SP_STATE_CODE                 int64
BENE_COUNTY_CD                int64
BENE_HI_CVRAGE_TOT_MONS       int64
BENE_SMI_CVRAGE_TOT_MONS      int64
BENE_HMO_CVRAGE_TOT_MONS      int64
PLAN_CVRG_MOS_NUM             int64
SP_ALZHDMTA                   int64
SP_CHF                        int64
SP_CHRNKIDN                   int64
SP_CNCR                       int64
SP_COPD                       int64
SP_DEPRESSN                   int64
SP_DIABETES                   int64
SP_ISCHMCHT                   int64
SP_OSTEOPRS                   int64
SP_RA_OA                      int64
SP_STRKETIA                   int64
MEDREIMB_IP                   int64
BENRES_IP                     int64
PPPYMT_IP                     int64
MEDREIMB_OP                   int64
BENRES_OP                   

In [28]:
total_payments = bene_df.MEDREIMB_IP + bene_df.MEDREIMB_OP + bene_df.MEDREIMB_CAR

In [29]:
bene_df['total_payments'] = total_payments

In [30]:
bene_pvt = bene_df.pivot_table(index='file_year', values=['BENE_HI_CVRAGE_TOT_MONS', 'total_payments'], aggfunc='sum')

In [31]:
bene_pvt = bene_pvt.reset_index()

In [32]:
def evaluate_pmpm(x):
    member_months = x['BENE_HI_CVRAGE_TOT_MONS']
    total_payments = x['total_payments']
    return total_payments / member_months * 12

In [33]:
bene_pvt.apply(evaluate_pmpm, axis=1)

0   4,866.23
1   5,060.01
2   2,594.32
dtype: float64

In [10]:
bene_pvt.to_excel('bene_pvt.xlsx')